In [182]:
import os
import sys
import numpy as np
import laspy as lp
import glob
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16
import shutil
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from scipy.interpolate import LinearNDInterpolator
from scipy.spatial import cKDTree
from tqdm import tqdm

In [183]:
input_ALS_folder = '/home/davidhersh/Dropbox/Uni/ThesisHersh/ALS_data'
output_folder = '/media/davidhersh/T7 Shield/pre-processing/tmp'

In [188]:
# k-fold
n_splits = 5

# Augmentation values
min_subsample_distance = 0.2
rotations = [-15, 15]

In [185]:
def copy_folder(input_folder, output_folder=output_folder, point_threshold=2000, plot=True, redo=False):
    """
    Copy single tree ALS files (ending with _g meaning ground classified) for a specific list of trees if they have a minimum point count
    """
    
    if redo:
    
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            
        # Save figs
        figdir = output_folder.replace('/tmp', '/processing_figures')
        if not os.path.exists(figdir):
            os.makedirs(figdir)
        
        # Only copy certain species
        species_to_copy = [
            "PseMen",
            "FagSyl",
            "PinSyl",
            "QueRub",
            "PicAbi",
            "QuePet"
        ]
    
        files = []
        species_counter = Counter()
        
        for root, dirs, filenames in os.walk(input_folder):
            for filename in filenames:
                try:
                    species_name = filename.split('_')[0]
        
                    # Study area BR06 has an issue with ground. Do not copy for further processing
                    # File format: FagSyl_BR02_04_2019-07-05_q2_ALS-on_g.laz
                    study_area = filename.split('_')[1] # BR02
        
                    if filename.endswith('ALS-on_g.laz') and species_name in species_to_copy and study_area != "BR06":
                            las = lp.read(os.path.join(root, filename))
                            number_of_nonground_points = len(las.points[las.classification !=2])
                            if number_of_nonground_points >= point_threshold:
                                species_counter[species_name] += 1
                                files.append(os.path.join(root, filename))  # Store full file paths
                                shutil.copyfile(os.path.join(root, filename), os.path.join(output_folder, filename))
                except: 
                    pass
        print(f'Copying {len(files)} files')
        print(f'Unique species: {species_counter}')
        
        if plot:
            fig, ax = plt.subplots(figsize=(12,8))
            plt.bar(species_counter.keys(), species_counter.values())
            plt.ylabel('Number of trees')
            plt.xlabel('Species')
            plt.title(f"Species with greater than {point_threshold} non-ground points")
            plt.savefig(os.path.join(figdir, f'species_count_greaterthan_{point_threshold}_points.png'), bbox_inches='tight')
            plt.show()
        
        return output_folder, species_counter
    
    else:
        print(f'Skipping copying...')


In [186]:
copied_als_folder, species_counter = copy_folder(input_ALS_folder)

Skipping copying...


TypeError: cannot unpack non-iterable NoneType object

In [187]:
def calculate_hag(las_file):
    las = lp.read(las_file)
    x = las.x
    y = las.y
    z = las.z
    
    classification = las.classification
    
    ground_indices = classification == 2
    x_ground = x[ground_indices]
    y_ground = y[ground_indices]
    z_ground = z[ground_indices]
    
    X, Y = np.meshgrid(x_ground, y_ground)
    ground_interpolated = LinearNDInterpolator(list(zip(x_ground, y_ground)), z_ground)
    Z = ground_interpolated(X, Y)
    z_ground_at_points = ground_interpolated(x, y)
    min_ground = np.nanmin(z_ground)
    z_ground_at_points = np.where(np.isnan(z_ground_at_points), min_ground, z_ground_at_points)
    hag = z - z_ground_at_points
    las.z = hag
    las = las[las.classification != 2]
    
    las.write(las_file)

In [145]:
# Can do HAG calculation before split

def convert_hag_to_z(input_folder=copied_als_folder, redo=False):
    
    if redo:
        files = os.listdir(copied_als_folder)
        for file in tqdm(files, desc='Processing files', unit='file'):
            full_path = os.path.join(copied_als_folder, file)
            calculate_hag(full_path)
    else:
        print("Skipping HAG step...")
        
convert_hag_to_z(input_folder=copied_als_folder)

Skipping HAG step...


In [175]:
def stratified_k_fold_split(input_folder, output_folder, n_splits=5):
    input_folder = copied_als_folder
    output_folder = input_folder.replace('/tmp', '/kfolders')

    files = []
    labels = []
    
    for file in os.listdir(input_folder):
        class_name = file.split('_')[0] #Fagsyl etc...
        files.append(os.path.join(input_folder, file))
        labels.append(class_name)
    class_counts = Counter(labels)
    print(f'Class counts: {class_counts}')
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=32)
    
    train_folders = []
    test_folders = []
    
    for fold, (train_index, test_index) in enumerate(skf.split(files, labels)):
        fold_train_dir = os.path.join(output_folder, f'fold_{fold+1}_train')
        train_folders.append(fold_train_dir)
        fold_test_dir = os.path.join(output_folder, f'fold_{fold+1}_val')
        test_folders.append(fold_test_dir)
        os.makedirs(fold_train_dir, exist_ok=True)
        os.makedirs(fold_test_dir, exist_ok=True)
        
        for train_idx in train_index:
            shutil.copy(files[train_idx], fold_train_dir)
        for test_idx in test_index:
            shutil.copy(files[test_idx], fold_test_dir)
            
    return train_folders, test_folders
    
train_folders, test_folders = stratified_k_fold_split(copied_als_folder, output_folder, n_splits=n_splits)

Class counts: Counter({'FagSyl': 294, 'PseMen': 185, 'PinSyl': 141, 'PicAbi': 121, 'QueRub': 104, 'QuePet': 75})


In [176]:
def poisson_subsample(las_file, min_distance=min_subsample_distance):
    las = lp.read(las_file)
    points = np.vstack((las.x, las.y, las.z)).transpose()

    kdtree = cKDTree(points)

    selected = np.zeros(len(points), dtype=bool)
    selected_indices = []
    for i, point in enumerate(points):
        if selected[i]:
            continue

        selected_indices.append(i)
        selected[i] = True

        indices = kdtree.query_ball_point(point, min_distance)
        selected[indices] = True

    header = lp.LasHeader(point_format=las.header.point_format, version=las.header.version)
    header.offsets = las.header.offsets
    header.scales = las.header.scales

    subsampled_las = lp.LasData(header)
    subsampled_las.points = las.points[selected_indices]

    subsampled_las.write(las_file)

In [177]:
def normalize_xy(las_file):
        las = lp.read(las_file)
        original_offset_x = las.header.offsets[0]
        original_offset_y = las.header.offsets[1]
        scale_x = las.header.scales[0]
        scale_y = las.header.scales[1]

        x = las.x
        y = las.y

        mean_x = np.mean(x)
        mean_y = np.mean(y)
        
        # Normalize coordinates
        normalized_x = x - mean_x
        normalized_y = y - mean_y

        new_header = lp.LasHeader(point_format=las.header.point_format, version=las.header.version)
        new_header.scales = las.header.scales

        new_header.offsets = [
            original_offset_x - mean_x,
            original_offset_y - mean_y,
            las.header.offsets[2]  # Same z
        ]

        new_las = lp.LasData(new_header)
        new_las.x = normalized_x
        new_las.y = normalized_y
        new_las.z = las.z

        for dim_name in las.point_format.dimension_names:
            if dim_name not in ["X", "Y", "Z"]:
                setattr(new_las, dim_name, getattr(las, dim_name))

        new_las.write(las_file)

In [178]:
def rotate_z(pointcloud, degrees):
    theta = np.deg2rad(degrees)

    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

    return np.dot(pointcloud, rotation_matrix.T)


def rotate_las(las_file, rotations):
        las = lp.read(las_file)

        point_data = np.vstack((las.x, las.y, las.z)).T

        for i, rotation in enumerate(rotations):

            rotated_points = rotate_z(point_data, rotation)

            header = lp.LasHeader(point_format=las.header.point_format, version=las.header.version)
            header.offsets = las.header.offsets
            header.scales = las.header.scales
            # header.crs = lp.crs.CRS.from_epsg(25832)

            rotated_las = lp.LasData(header)
            rotated_las.x = rotated_points[:, 0]
            rotated_las.y = rotated_points[:, 1]
            rotated_las.z = rotated_points[:, 2]

            for dim_name in las.point_format.dimension_names:
                if dim_name not in ["X", "Y", "Z"]:
                    setattr(rotated_las, dim_name, getattr(las, dim_name))

            unique_filename = las_file.replace('.laz', f'_rot_{rotation}.laz')
            output_filename = os.path.join(output_folder, unique_filename)

            os.makedirs(output_folder, exist_ok=True)
            rotated_las.write(output_filename)

In [181]:
# Augment each folder

def augmentation(train_folders=train_folders, test_folders=test_folders, 
                 redo_xy_normalization=False,
                 redo_subsample=False,
                 redo_rotation=False):

    all_folders = train_folders + test_folders
    for folder in all_folders:
        for las_file in os.listdir(folder):
            
            # Normalize x,y by means
            if redo_xy_normalization:
                normalize_xy(las_file=os.path.join(folder, las_file))
            else:
                pass
            
            # Downsample using minimum distance
            if redo_subsample:
                poisson_subsample(las_file=os.path.join(folder, las_file),
                                  min_distance=min_subsample_distance)
            else:
                pass
            
            # Rotate on z-axis
            if redo_rotation:
                rotate_las(os.path.join(folder, las_file), rotations=rotations)
            else:
                pass
    
augmentation()

In [ ]:
# Convert to KPConv repository dataset format

# Loop over each dataset and run trainNeuesPalaisTrees.py
# Need to pass config parameters here and update class_w